In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import ccxt
from sqlalchemy import create_engine
import os
from datetime import datetime, timedelta


with open("../api(JKBOT).txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, #"XTpKrQGSk3GhXzqiEV4OfwGJzmTVcLh8dKGwHo4aQBH4p0mOqPDpIsxdh95tjGVf",
    'secret': secret, #"A0eqZGEWHsyL3NMM6WuDrucIanr7A2YZAnrwXVPhXpf2WGauIANwa5zsoNeNt0hs",
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})

jongmok= []

markets = binance.load_markets()
for market in markets.keys():
    if market.endswith("/USDT"):# and market != "BCC/USDT" and market != "TUSD/USDT":
        jongmok.append(market)

database = 'price_1d'

def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

engine = md_connect('root', '1234', database, 'localhost')

print("database :" + database + " 에 접속중 입니다")

score = []
window_list = []
away_list = []
size=30
term = '1d'
window = 28
away = 7
# # for away in range(1, 14):
# print("window : " + str(window), "away : " + str(away))


database :price_1d 에 접속중 입니다


In [2]:
#모델 만들기

x_input = []
y_target = []
for symbol in jongmok[0:80]:
    data = pd.read_sql(symbol[:-5].lower(), engine, index_col='datetime') #sql에서 db 받아오기
    x_data = data[['open', 'high', 'low', 'close']]
    y_data = data[['close']]
    print(symbol + " Data Collect")
    for i in range(0, len(data) - window - away + 1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        # x['volume'] = (x_data['volume'] - volume_mean) / (volume_std)
        x_input.append(x)

        if y_data['close'][i+window-1] < y_data['close'][i+window+away-1]:
            y_target.append(1)
        else:
            y_target.append(0)

x_input = np.array(x_input)
y_target = np.array(y_target)

train_input, test_input, train_target, test_target = train_test_split(x_input, y_target, test_size=0.5)
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(size, activation='relu', input_shape=(window, 4), dropout=0.3))
# model.add(keras.layers.LSTM(10, activation = 'relu', input_shape=(window, 4)))
# model.add(keras.layers.Dropout(0, 1))
# model.add(keras.layers.LSTM(8))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5', monitor='val_loss', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
epochs = 100
print("window : " + str(window), "away : " + str(away), "에 대해 "+ str(epochs) +"회 학습중")

model.fit(train_input, train_target, validation_data=(test_input, test_target), epochs=epochs, verbose=1, callbacks=[checkpoint_cb, early_stopping_cb])
print(window, away, model.evaluate(test_input, test_target)[1])

away_list.append([window, away, model.evaluate(test_input, test_target)[1]])
tot_score = pd.DataFrame(away_list)
with pd.ExcelWriter('excel/' + term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').xlsx') as writer:
    tot_score.to_excel(writer, index=False)



NameError: name 'jongmok' is not defined

In [2]:
from sqlalchemy import create_engine
# sql 연결 함수
def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

# 백테스팅 결과 함수
def select_rate(long_std, short_std, start_date=0, end_date=0, fee=0):
    df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
    condition = df['ind'] > 0.5
    df = df.assign(position=condition.map({True:1, False:-1}))
    df['profit'] = df['position'] * df['rate'] -(2 * fee)

    if start_date != 0:
        if end_date != 0:
            df = df.loc[start_date:end_date]
        else:
            df = df.loc[start_date:]
    else:
        if end_date !=0:
            df = df.loc[:end_date]

    df['count_sum'] = df.groupby('datetime').count()['sym'].rolling(window=7, min_periods=1).sum()
    df['tot_profit'] = df['profit'].cumsum()

    return df

log_column = ['시작', '끝', '거래기간', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)

engine = md_connect('root', '1234', 'price_1d', 'localhost')

# for window in range(7, 35, 7):
#     for away in range(3, 8):

#가격데이터 연결

account = [] # 계좌 정보
diary = [] # 매매일지

tot_data = {}

# 가격데이터 가져오기
for sym in jongmok[:80]:
    sym = sym[:-5].lower()
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

# predict 모델 가져오기
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5')

engine = md_connect('root', '1234', 'ind_test', 'localhost')

# 예측하기
for key in jongmok[:80]:
    key = key[:-5].lower()
    data = tot_data[key]
    x_data = tot_data[key][['open', 'high', 'low', 'close']]
    y_data = tot_data[key][['close']]

    x_input = []
    y_input = []

    ## 전처리
    for i in range(len(data)-window-away+1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        x_input.append(x)
    #
    ## ind 지표 생성
    x_input = np.array(x_input)
    predict_data = model.predict(x_input).tolist()

    ## 수익률 생성
    data['rate'] = data['close'].pct_change(periods=away).shift(-away)

    predict = []
    for i in range(len(predict_data)):
        predict.append(predict_data[i][0])
    data = data[window-1:-away]

    ##ind 지표 결합
    data = data.copy()
    data['ind'] = predict
    data['sym'] = key

    data = data[['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']]
    data.to_sql(key, engine, if_exists="replace", index=True)

    

24/24 [==============================] - 0s 1ms/step


In [3]:
engine = md_connect('root', '1234', 'ind_test', 'localhost')

for sym in jongmok[:40]:
    sym = sym[:-5].lower()
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

    # 변수 설정
ind_data = []
for key in jongmok[:40]:
    key = key[:-5].lower()
    ind_data.append(tot_data[key][['sym', 'close', 'rate', 'ind']])

ind_data = pd.concat(ind_data)

In [6]:
import matplotlib.pyplot as plt
start_date = '2023-03-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
# end_date = start_date + timedelta(days=30)
end_date = '2023-03-20'
end_date = datetime.strptime(end_date, '%Y-%m-%d')
# end_date = start_date + timedelta(days=30)
# for i in range(22):
long_std = 0.8
short_std = 0.20

df = select_rate(long_std, short_std, start_date, end_date, 0.0004)

win_cnt = (df['profit'] > 0).sum()
lose_cnt = (df['profit'] < 0).sum()

day = end_date - start_date
day.days

# log.append([len(df), day.days, win_cnt / len(df), win_cnt, lose_cnt, df['count_sum'].mean(), df['profit'].mean(), df['profit'].sum()])
log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수' : len(df), '거래기간' : day.days, '승률' : win_cnt / len(df), '승수' : win_cnt, '패수' : lose_cnt, '평균보유량' : df['count_sum'].mean(), '평균수익률' : df['profit'].mean(), '총수익률' : df['profit'].sum()}
log_df = log_df.append(log, ignore_index=True)

# start_date = end_date + timedelta(days=1)
# end_date = end_date + timedelta(days=31)
# with pd.ExcelWriter('log/log_df'+str(away) +str(window) + '.xlsx') as writer:
#     log_df.to_excel(writer, index=False)

C:\Users\tomkj\AppData\Local\Temp\ipykernel_13096\1262881476.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)


In [31]:
away
window

28

In [7]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,window,away
0,2022-11-01,2022-12-20,49,0.8,0.2,153,0.915033,140,13,35.751634,0.094767,14.499302,28.0,7.0
1,2023-03-01,2023-03-20,19,0.8,0.2,37,0.783784,29,8,12.324324,0.067768,2.507420,28.0,7.0


In [9]:
select_rate(long_std, short_std, start_date, end_date, 0.0004)

,sym,close,rate,ind,position,profit,count_sum,tot_profit
datetime,,,,,,,,
2021-01-01,ada,0.17517,0.737113,0.839722,1,0.736313,32.0,0.736313
2021-01-01,eos,2.64000,0.178788,0.881206,1,0.177988,32.0,0.914300
2021-01-01,rlc,0.74830,0.554590,0.963724,1,0.553790,32.0,1.468091
2021-01-01,defi,662.40000,0.365640,0.928610,1,0.364840,32.0,1.832931
2021-01-01,bat,0.20460,0.227761,0.968000,1,0.226961,32.0,2.059892
...,...,...,...,...,...,...,...,...
2022-12-30,btc,16601.20000,0.020637,0.238900,-1,-0.021437,21.0,1165.529296
2022-12-31,xmr,147.24000,0.057797,0.769822,1,0.056997,24.0,1165.586293
2022-12-31,zrx,0.14760,0.072493,0.756778,1,0.071693,24.0,1165.657986


In [10]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,away,window
0,2021-01-01,2022-12-31,729,0.75,0.25,10807,0.825761,8924,1883,185.059313,0.107866,1165.710346,7.0,28.0


In [11]:
df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
df

,sym,close,rate,ind
datetime,,,,
2019-10-17,btc,8069.20000,-0.081526,0.202355
2019-10-18,btc,7943.94000,0.088880,0.204504
2019-10-19,btc,7947.13000,0.161705,0.180813
2019-10-20,btc,8218.23000,0.160021,0.163228
2019-10-21,btc,8201.74000,0.123830,0.153966
...,...,...,...,...
2023-03-10,snx,2.23800,0.357462,0.756991
2023-03-11,eos,0.99300,0.104733,0.827663
2023-03-11,ren,0.07737,0.211322,0.934642


In [ ]:
import keras
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import matplotlib
from datetime import datetime, timedelta

def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

engine = md_connect('root', '1234', 'price_1d', 'localhost')



In [ ]:
account = [] # 계좌 정보
diary = [] # 매매일지

window = 14
away = 7

start_date = ''
end_date = ''

tot_data = {}
engine = md_connect('root', '1234', 'price_1d', 'localhost')
for sym in jongmok[:100]:
    data = pd.read_sql(sym[:-5].lower(), engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

In [ ]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5')

engine = md_connect('root', '1234', 'ind_test', 'localhost')

for key in jongmok[:100]:
    data = tot_data[key]
    x_data = tot_data[key][['open', 'high', 'low', 'close']]
    y_data = tot_data[key][['close']]

    x_input = []
    y_input = []

    ## 전처리
    for i in range(len(data)-window-away+1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        x_input.append(x)
    #
    ## ind 지표 생성
    x_input = np.array(x_input)
    predict_data = model.predict(x_input).tolist()

    ## 수익률 생성
    data['rate'] = data['close'].pct_change(periods=away).shift(-away)


    predict = []
    for i in range(len(predict_data)):
        predict.append(predict_data[i][0])
    data = data[window-1:-away]

    ##ind 지표 결합
    data['ind'] = predict
    print(len(data))

    data['sym'] = key


    data = data[['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']]


    data.to_sql(key, engine, if_exists="replace", index=True)

In [ ]:
engine = md_connect('root', '1234', 'ind_test', 'localhost')
for sym in jongmok[:80]:
    data = pd.read_sql(sym[:-5].lower(), engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

In [ ]:
# 변수 설정
ind_data = []
for key in jongmok[:80]:
    ind_data.append(tot_data[key][['sym', 'close', 'rate', 'ind']])

ind_data = pd.concat(ind_data)


def select_rate(long_std, short_std, start_date=0, end_date=0, fee=0):
    df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
    condition = df['ind'] > 0.5
    df = df.assign(position=condition.map({True:1, False:-1}))
    df['profit'] = df['position'] * df['rate'] -(2 * fee)

    if start_date != 0:
        if end_date != 0:
            df = df.loc[start_date:end_date]
        else:
            df = df.loc[start_date:]
    else:
        if end_date !=0:
            df = df.loc[:end_date]


    df['count_sum'] = df.groupby('datetime').count()['sym'].rolling(window=7, min_periods=1).sum()
    df['tot_profit'] = df['profit'].cumsum()

    return df

In [ ]:
ind_data

In [ ]:
log_column = ['시작', '끝', '거래기간', 'window', 'away', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)

In [8]:
import matplotlib.pyplot as plt
start_date = '2021-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = start_date + timedelta(days=30)

for i in range(22):
    long_std = 0.8
    short_std = 0.2

    df = select_rate(long_std, short_std, start_date, end_date, 0.0004)

    win_cnt = (df['profit'] > 0).sum()
    lose_cnt = (df['profit'] < 0).sum()

    day = end_date - start_date
    day.days

    # log.append([len(df), day.days, win_cnt / len(df), win_cnt, lose_cnt, df['count_sum'].mean(), df['profit'].mean(), df['profit'].sum()])
    log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수' : len(df), '거래기간' : day.days, '승률' : win_cnt / len(df), '승수' : win_cnt, '패수' : lose_cnt, '평균보유량' : df['count_sum'].mean(), '평균수익률' : df['profit'].mean(), '총수익률' : df['profit'].sum()}
    log_df = log_df.append(log, ignore_index=True)

    start_date = end_date + timedelta(days=1)
    end_date = end_date + timedelta(days=31)



C:\Users\tomkj\AppData\Local\Temp\ipykernel_13096\4230835338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_13096\4230835338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_13096\4230835338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_13096\4230835338.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)


In [9]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,window,away
0,2022-11-01,2022-12-20,49,0.8,0.2,153,0.915033,140,13,35.751634,0.094767,14.499302,28.0,7.0
1,2023-03-01,2023-03-20,19,0.8,0.2,37,0.783784,29,8,12.324324,0.067768,2.507420,28.0,7.0
2,2021-01-01,2021-01-31,30,0.8,0.2,67,0.805970,54,13,22.791045,0.196220,13.146713,28.0,7.0
3,2021-02-01,2021-03-03,30,0.8,0.2,54,1.000000,54,0,29.648148,0.412802,22.291303,28.0,7.0
4,2021-03-04,2021-04-03,30,0.8,0.2,68,0.838235,57,11,19.529412,0.135195,9.193249,28.0,7.0
5,2021-04-04,2021-05-04,30,0.8,0.2,19,0.789474,15,4,9.631579,0.454949,8.644032,28.0,7.0
6,2021-05-05,2021-06-04,30,0.8,0.2,62,0.838710,52,10,27.967742,0.261225,16.195958,28.0,7.0
7,2021-06-05,2021-07-05,30,0.8,0.2,404,0.920792,372,32,139.623762,0.155042,62.637048,28.0,7.0
8,2021-07-06,2021-08-05,30,0.8,0.2,188,0.898936,169,19,58.601064,0.119533,22.472214,28.0,7.0
9,2021-08-06,2021-09-05,30,0.8,0.2,6,0.666667,4,2,4.166667,0.130568,0.783410,28.0,7.0


In [ ]:
start_date = '2022-01-01'

In [ ]:
from datetime import datetime, timedelta

In [ ]:
start_date = datetime.strptime(start_date, '%Y-%m-%d')

In [ ]:
start_date = start_date + timedelta(days=90)

In [ ]:
start_date